In [1]:
!pip install nltk transformers sentencepiece -q


In [2]:
import nltk
from nltk import word_tokenize, pos_tag
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [5]:
checkpoint = 'facebook/nllb-200-3.3B'

In [6]:
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [12]:
source_lang = "eng_Latn"
target_lang = "hin_Deva"


In [13]:
translator = pipeline('translation', model=model, tokenizer=tokenizer, src_lang=source_lang, tgt_lang=target_lang, device=0, max_length = 400)

In [14]:
#function that returns nouns
def return_nouns(text : str) -> list:
    tokens = word_tokenize(text)
    parts_of_speech = nltk.pos_tag(tokens)
#     print(parts_of_speech)
    nouns = list(filter(lambda x: x[1] == "NN" or x[1] == "NNS" or x[1] == "NNP" or x[1] == "NNPS", parts_of_speech))
    
    return nouns

In [15]:
#map key value pairs
def key_value_pair(nouns: list) -> dict:
    temp_dict = dict()
    for noun in nouns:
        value = translator(noun[0]+".")[0]['translation_text']
        if value[-1] == "." or value[-1] == "।":
            value = value[:-1]
        temp_dict[noun[0]] = value
        
    return temp_dict

In [17]:
#function for converting hindi text to hinglish
def hindi_to_hinglish(en_text: str, hi_text: str) -> str:
    #find nouns from english text
    nouns = return_nouns(en_text)
    #pair it with its hindi translation
    en_hi_transliteration = key_value_pair(nouns)
    
    hindi_text = hi_text.split()
    
    for k, v in en_hi_transliteration.items():
        for i in range(len(hindi_text)):
            if v in hindi_text[i]:
                hindi_text[i] = k
                
    return " ".join(hindi_text)

In [18]:
#main pipeline function
def english_to_hinglish(en_text: str) -> str:
    hi_text = translator(en_text)[0]['translation_text']
    hinglish_text = hindi_to_hinglish(en_text, hi_text)
    
    return hinglish_text

In [19]:
english_to_hinglish("Definitely share your feedback in the comment section.")

'comment अनुभाग में अपनी feedback अवश्य साझा करें।'

In [20]:
english_to_hinglish("So even if it's a big video, I will clearly mention all the products.")

'तो भले ही यह एक बड़ा video हो, मैं स्पष्ट रूप से सभी products का उल्लेख करूंगा।'

In [30]:
english_to_hinglish("I was waiting for my bag.")

'मैं अपने bag की प्रतीक्षा कर रहा था।'

In [31]:
english_to_hinglish("I had about a 30 minute demo just using this new headset.")

'मैंने इस नए headset का उपयोग करते हुए लगभग 30 मिनट का demo किया।'